<!-- SPDX-License-Identifier: CC-BY-4.0 -->
<!-- Copyright Contributors to the ODPi Egeria project 2024. -->

![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/main/assets/img/ODPi_Egeria_Logo_color.png)

### Egeria Workbook

# Working with pyegeria

## Introduction

The `pyegeria` python library provides a wide-ranging set of python functions for interacting with the Egeria runtime.  In addition there is a command level interface (CLI) for issuing commands to Egeria and widgets to display information held by Egeria.

This notebook provides an introductory guide to `pyegeria`.

## Ensuring you have the latest level of pyegeria

Pyegeria is installed in this JupyterLab environment.  However, it is advancing rapidly and it is worthwhile checking that you have the latest level.  Firstly, to upgrade the base python functions, run the following command:

-----

In [1]:
pip install pyegeria --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 MB 6.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pyegeria
    Found existing installation: pyegeria 5.2.0.42.6
    Uninstalling pyegeria-5.2.0.42.6:
      Successfully uninstalled pyegeria-5.2.0.42.6
Note: you may need to restart the kernel to use updated packages.


----
If the libraries are upgraded, you will need to restart the python kernel.  From the JupyterLab menubar select `Kernel->Restart Kernel...` and then confirm that you want the kernel to restart.

The next step is to check that the widgets and CLI are up-to-date.  For this you need a Terminal window.  Click on the plus (+) to open a new tab.  This displays the launcher page.  Then click on *Terminal* to open up the Terminal window.

----

![Launcher](images/launcher.png)

----

From the Terminal window enter the command `pipx upgrade pyegeria` command.   This will upgrade the CLI if required.

```
$ pipx upgrade pyegeria
Upgraded package pyegeria from 0.7.45 to 0.8.4.1 (location: /home/jovyan/.local/share/venvs/pipx/pyegeria)
$

```

As a quick check to ensure the CLI is correctly installed, enter `pipx list` in the terminal window.  This will show all of the commands available:

```
$ pipx list
venvs are in /home/jovyan/.local/share/pipx/venvs
apps are exposed on your $PATH at /home/jovyan/.local/bin
manual pages are exposed at /home/jovyan/.local/share/man
   package pyegeria 0.8.4.1, installed using Python 3.11.9
    - change_todo_status
    - create_todo
    - delete_todo
    - get_asset_graph
    - get_collection
    - get_element_info
    - get_guid_info
    - get_project_dependencies
    - get_project_structure
    - get_tech_details
    - get_tech_type_elements
    - get_tech_type_template
    - hey_egeria
    - hey_egeria_cat
    - hey_egeria_my
    - hey_egeria_ops
    - hey_egeria_tech
    - list_archives
    - list_asset_types
    - list_assets
    - list_catalog_targets
    - list_cert_types
    - list_elements
    - list_engine_activity
    - list_engine_activity_compressed
    - list_glossary
    - list_gov_eng_status
    - list_integ_daemon_status
    - list_my_profile
    - list_projects
    - list_registered_services
    - list_related_specification
    - list_relationship_types
    - list_relationships
    - list_tech_templates
    - list_tech_types
    - list_todos
    - list_user_ids
    - list_valid_metadata_values
    - load_archive
    - load_archive_tui
    - mark_todo_complete
    - monitor_asset_events
    - monitor_coco_status
    - monitor_engine_activity
    - monitor_engine_activity_compressed
    - monitor_gov_eng_status
    - monitor_integ_daemon_status
    - monitor_my_todos
    - monitor_open_todos
    - monitor_platform_status
    - monitor_server_list
    - monitor_server_status
    - reassign_todo
    - refresh_integration_daemon
    - restart_integration_daemon
    - start_daemon
    - stop_daemon
$ 
```

## Using the pyegeria python functions

The `pyegeria` functions each have both a synchronous and an asynchronous version.  The asynchronous versions are used in the widgets, the synchronous versions use the asynchronous methods under the covers.  In order to avoid clashes between Egeria's thread management and JupyterLab's operation you need to add the following code to your nottebook before calling any `pyegeria` functions.

----

In [1]:

import asyncio
import nest_asyncio
nest_asyncio.apply()


----

The python functions are organized into 5 separate clients:

* **EgeriaMy** - for querying an individual's profiles, to-dos and adding feedback to metadata stored in Egeria.
* **EgeriaCat** - for day-to-day use of Egeria
* **EgeriaTech** - for setting up Egeria services and governance actions.
* **EgeriaGov (coming)** - for creating and maintaining governance domains.
* **EgeriaConfig** - for configuring new [Egeria OMAG Servers](https://egeria-project.org/concepts/omag-server/)

`EgeriaCat`, `EgeriaTech` and `EgeriaGov` all extend `EgeriaMy`. `EgeriaConfig` is standalone.

To make use of one of these clients, first import the client you need.  The command below imports `EgeriaTech`.

----

In [2]:

from pyegeria import EgeriaTech


----

The `pyegeria` functions require information about the calling user and the network location of Egeria's runtime.  These are passed to the client when it is created.  The CLI also needs this information and to save the caller having to supply it on every request, it supports environment variables.  These same environment variables can be used with the `pyegeria` functions.

The code below extracts the values from these environment variables for use in the call to the constructor of the client.

----

In [4]:

import os

view_server = os.environ.get("VIEW_SERVER","view-server")
url = os.environ.get("EGERIA_VIEW_SERVER_URL","https://localhost:9443")
user_id = os.environ.get("EGERIA_USER", "peterprofile")
user_pwd = os.environ.get("EGERIA_USER_PASSWORD")


----

### EgeriaTech help

This is the call to create the client.

----

In [5]:
egeria_tech = EgeriaTech(view_server, url, user_id, user_pwd)

-----

The next call logs the user into the Egeria platform, a token is returned.  It is retained by the client and automatically passed on each request to Egeria.  The token will eventually time out, at which time just reissue the `create_egeria_bearer_token()` function.

-----

In [6]:
egeria_tech_token = egeria_tech.create_egeria_bearer_token()

----

Now you are ready to use the functions that call Egeria.  It is possible to use the standard python methods to retrieve information about the available functions. For example `dir` lists the functions supported by the client.  

----

In [7]:
dir(egeria_tech)

['__async_get_guid__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__get_guid__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__validate_term_status__',
 '__weakref__',
 '_async_activate_server_with_stored_config',
 '_async_add_archive_content',
 '_async_add_archive_file',
 '_async_add_catalog_target',
 '_async_add_comment_reply',
 '_async_add_comment_to_element',
 '_async_add_confidentiality_to_term',
 '_async_add_data_field_to_term',
 '_async_add_element_to_subject_area',
 '_async_add_gov_definition_to_element',
 '_async_add_like_to_element',
 '_async_add_ownership_to_element',
 '_async_add_rating_to_element',
 '_async_add_subject_area_to_term',
 '_async_add_tag_to_element',


----

A question mark (?) after the name of the client shows how to create an instance of the client.

----

In [6]:
EgeriaTech?

Init signature:
EgeriaTech(
    view_server: str,
    platform_url: str,
    user_id: str,
    user_pwd: str = None,
    token: str = None,
)
Docstring:     
Client for technical Egeria users.

Attributes:

    view_server: str
            Name of the server to use.
    platform_url : str
        URL of the server platform to connect to
    user_id : str
        The identity of the user calling the method - this sets a default optionally used by the methods
        when the user doesn't pass the user_id on a method call.
    user_pwd: str
        The password associated with the user_id. Defaults to None
    token: str, optional
        Bearer token

Methods:
    Inherits methods from EgeriaCat, ActionAuthor, AutomatedCuration,
    ClassificationManager, RegisteredInfo, ValidMetadataManager
File:           /opt/conda/lib/python3.12/site-packages/pyegeria/egeria_tech_client.py
Type:           type
Subclasses:     

----

It is then possible to list the parameters of a specific function using the `help` function.

----

In [7]:
help(EgeriaTech.get_related_elements)

Help on function get_related_elements in module pyegeria.classification_manager_omvs:

get_related_elements(self, element_guid: str, relationship_type: str = None, open_metadata_type_name: str = None, start_at_end: int = 1, effective_time: str = None, for_lineage: bool = None, for_duplicate_processing: bool = None, start_from: int = 0, page_size: int = 500, time_out: int = 30) -> list | str
    Retrieve elements linked by relationship type name. If the relationship type is None, then all related elements
    will be returned. It is also possible to limit the results by specifying a type name for the elements that
    should be returned. If no type name is specified then any type of element may be returned.

    https://egeria-project.org/types/

    Parameters
    ----------
    element_guid: str
        - the base element to get related elements for
    relationship_type: str
        - the type of relationship to navigate to related elements
    open_metadata_type_name : str, default 

----

If you execute a function without capturing the response in a python variable, the JSON returned from Egeria is displayed:

----

In [8]:

egeria_tech.get_servers_by_name("active-metadata-store")


[{'properties': {'class': 'SoftwareServerProperties',
   'typeName': 'SoftwareServer',
   'qualifiedName': 'Metadata Access Server:active-metadata-store',
   'displayName': 'active-metadata-store',
   'displayDescription': 'A metadata store that supports Open Metadata Access Services (OMASs) with event notifications.  It provides metadata to view-server, engine-host and integration-daemon.',
   'name': 'active-metadata-store',
   'resourceName': 'active-metadata-store',
   'versionIdentifier': 'Egeria OMAG Server Platform (version 5.2-SNAPSHOT)',
   'resourceDescription': 'A metadata store that supports Open Metadata Access Services (OMASs) with event notifications.  It provides metadata to view-server, engine-host and integration-daemon.',
   'deployedImplementationType': 'Metadata Access Server'},
  'elementHeader': {'class': 'ElementHeader',
   'headerVersion': 0,
   'status': 'ACTIVE',
   'type': {'typeId': 'aa7c7884-32ce-4991-9c41-9778f1fec6aa',
    'typeName': 'SoftwareServer',
 

----

Assigning the results to a python variable means you can extract specific values:

-----

In [10]:

def get_server_guid(serverName):
    serverDetails = egeria_tech.get_servers_by_name(serverName)
    if serverDetails:
        elementHeader = serverDetails[0].get('elementHeader')
        if elementHeader:
            return elementHeader.get('guid')
    return "???"
            

serverGUID = get_server_guid("active-metadata-store")

print("Unique identifier (guid) for 'active-metadata-store' is " + serverGUID)

Unique identifier (guid) for 'active-metadata-store' is 492a3052-737c-4c28-bb42-090661b2ea39


----

### EgeriaCat help

This is the call to list the functions for EgeriaCat.

----

In [ ]:
from pyegeria import EgeriaCat

egeria_cat = EgeriaCat(view_server, url, user_id, user_pwd)
egeria_cat_token = egeria_cat.create_egeria_bearer_token()

dir(egeria_cat)

----

Next is the help request for `get_asset_graph`.

-----

In [ ]:

help(EgeriaCat.find_assets_in_domain)


----

### EgeriaConfig help

This is the call to list the functions for EgeriaConfig.

----

In [ ]:
from pyegeria import EgeriaConfig

egeria_config = EgeriaConfig(view_server, url, user_id, user_pwd)
egeria_config_token = egeria_tech.create_egeria_bearer_token()

dir(egeria_tech)

----

Next is the help request for `find_elements_by_property_value`.

-----

In [ ]:

help(EgeriaConfig.get_known_servers)


In [ ]:
from pyegeria.commands.cat.glossary_actions import display_glossaries


In [ ]:
# print("*", view_server, url, user_id, user_pwd)
display_glossaries(search_string="*", view_server = view_server, url = url, uuser_id=user_id, user_pwd = user_pwd)


In [ ]:
help(list_glossaries)

In [6]:
from pyegeria.commands import display_gov_eng_status


ImportError: cannot import name 'display_gov_eng_status' from 'pyegeria.commands' (/opt/conda/lib/python3.11/site-packages/pyegeria/commands/__init__.py)